In [2]:
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import numpy as np
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import cv2
import numpy as np
import pickle
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.python.ops import math_ops
from Hourglass import *
from PIL import Image

In [3]:
def res_block_gen(model, kernal_size, filters, strides,name,SR=None) :
    
    gen = model
    if SR:
        model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same",name=name+'_1conv',weights=SR.get_layer(name+'_1conv').get_weights())(model)
        model = BatchNormalization(name=name+'_1bn',weights=SR.get_layer(name+'_1bn').get_weights())(model)
    else:
        model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same",name=name+'_1conv')(model)
        model = BatchNormalization(name=name+'_1bn')(model)
    model = Activation('relu',name=name+'_act')(model)
    if SR:
        model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same",name=name+'_2conv',weights=SR.get_layer(name+'_2conv').get_weights())(model)
        model = BatchNormalization(name=name+'_2bn',weights=SR.get_layer(name+'_2bn').get_weights())(model)
    else:
        model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same",name=name+'_2conv')(model)
        model = BatchNormalization(name=name+'_2bn')(model)
    model = add([gen, model],name=name+'_add')
    
    return model

def generator(gen_input,SR=None):
        
#     gen_input = Input(shape = noise_shape)
    if SR:
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C1_feature_extract",weights=SR.get_layer("C1_feature_extract").get_weights())(gen_input)
    else:
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C1_feature_extract")(gen_input)
    model = Activation('relu',name="C1_RELU")(model)
    
    for index in range(12):
        model= res_block_gen(model, 3, 64, 1,"C1_%d"%index,SR)
    
    
    model = BatchNormalization(name="C1_Encode_BN")(model)
    model = Activation('relu',name="C1_Encode_End")(model)
    model = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C1_Decode_End")(model)
    return model

def psnr(y_true,y_pred):
    if(len(y_true.shape)<=2):
        return 0 
    if(y_pred.shape[3] == 5):
        return 0
    img1 = tf.matmul(y_true, [[65.481], [128.553], [24.966]]) / 255.0 + 16.0
    img2 = tf.matmul(y_pred, [[65.481], [128.553], [24.966]]) / 255.0 + 16.0
    mse = math_ops.reduce_mean(math_ops.squared_difference(img1,img2), [-3, -2, -1])
    def log10(x):
        numerator = tf.compat.v1.log(x)
        denominator = tf.compat.v1.log(tf.constant(10, dtype=numerator.dtype))
        return numerator / denominator
    return 10*log10(255.0*255.0/mse)

def superFAN():

    input_img = Input(shape=(24,24,3,),name='C1_Input')
    upsample_img2 = Input(shape=(48,48,3,),name='C1_Upsample')
    upsample_img4 = Input(shape=(96,96,3,),name='C2_Upsample')
    upsample_img8 = Input(shape=(192,192,3,),name='C3_Upsample')
    
    upsample = generator(input_img)
    LR2 = Conv2D(3,3 , padding='same', kernel_initializer='he_normal',name='C1_RESIDUAL')(upsample)
    LR2_ = add([LR2,upsample_img2],name='C1_RGB')
   

    for index in range(3):
        upsample = res_block_gen( upsample, 3, 64, 1,"C2_%d"%index)
    upsample = BatchNormalization(name="C2_Encode_BN")(upsample)
    upsample = Activation('relu',name="C2_Encode_End")(upsample)
    upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C2_Decode_End")(upsample)
    
    LR4 = Conv2D(3,3, padding='same', kernel_initializer='he_normal',name='C2_RESIDUAL')(upsample)
    LR4_ = add([LR4,upsample_img4],name='C2_RGB')
   

    for index in range(3):
        upsample = res_block_gen( upsample, 3, 64, 1,"C3_%d"%index)
    upsample = BatchNormalization(name="C3_Encode_BN")(upsample)
    upsample = Activation('relu',name="C3_Encode_End")(upsample)
    upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C3_Decode_End")(upsample)
    
    upsample = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C3_conv1")(upsample)
    upsample = Activation('relu',name="C3_act1")(upsample)
    upsample = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C3_conv2")(upsample)
    upsample = Activation('relu',name="C3_act2")(upsample)
    
    ##这里的conv kernel size变成了3 
    LR8 = Conv2D(3,3, padding='same', kernel_initializer='he_normal',name='C3_RESIDUAL')(upsample)
    LR8_ = add([LR8,upsample_img8],name='C3_RGB')
    # LR8 = PReLU()(LR8)
    # LR8 = Conv2D(3,3, padding='same')(LR8)
    out = []

    out.append(LR2_)
    out.append(LR4_)
    out.append(LR8_)
    
    return Model([input_img,upsample_img2,upsample_img4,upsample_img8], out)

def superFAN_Concat(FAN1=None,FAN2=None):
    input_img = Input(shape=(24,24,3,),name='C1_Input')
    upsample_img2 = Input(shape=(48,48,3,),name='C1_Upsample')
    upsample_img4 = Input(shape=(96,96,3,),name='C2_Upsample')
    upsample_img8 = Input(shape=(192,192,3,),name='C3_Upsample')
    
    upsample = generator(input_img)
    LR2 = Conv2D(3,3 , padding='same', kernel_initializer='he_normal',name='C1_RESIDUAL')(upsample)
    LR2_ = add([LR2,upsample_img2],name='C1_RGB')
   

    for index in range(3):
        upsample = res_block_gen( upsample, 3, 64, 1,"C2_%d"%index)
    LR2 = create_hourglass_network(1,64,bottleneck_block,8,FAN1)(LR2)
    upsample = Concatenate(name='C2_Concat')([upsample,LR2])
    upsample = BatchNormalization(name="C2_Encode_BN")(upsample)
    upsample = Activation('relu',name="C2_Encode_End")(upsample)
    
    
    upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C2_Decode_End")(upsample)
    
    LR4 = Conv2D(3,3, padding='same', kernel_initializer='he_normal',name='C2_RESIDUAL')(upsample)
    LR4_ = add([LR4,upsample_img4],name='C2_RGB')
   

    for index in range(3):
        upsample = res_block_gen( upsample, 3, 64, 1,"C3_%d"%index)
    LR4 = create_hourglass_network(1,64,bottleneck_block,8,FAN2)(LR4)
    upsample = Concatenate(name='C3_Concat')([upsample,LR4])
    upsample = BatchNormalization(name="C3_Encode_BN")(upsample)
    upsample = Activation('relu',name="C3_Encode_End")(upsample)
    
    
    
    upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C3_Decode_End")(upsample)
    
    upsample = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C3_conv1")(upsample)
    upsample = Activation('relu',name="C3_act1")(upsample)
    upsample = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C3_conv2")(upsample)
    upsample = Activation('relu',name="C3_act2")(upsample)
    
    ##这里的conv kernel size变成了3 
    LR8 = Conv2D(3,3, padding='same', kernel_initializer='he_normal',name='C3_RESIDUAL')(upsample)
    LR8_ = add([LR8,upsample_img8],name='C3_RGB')
    # LR8 = PReLU()(LR8)
    # LR8 = Conv2D(3,3, padding='same')(LR8)
    out = []
#     out.append(LR2)
#     out.append(LR4)
#     out.append(LR8)
    out.append(LR2_)
    out.append(LR4_)
    out.append(LR8_)
    
 
    return Model([input_img,upsample_img2,upsample_img4,upsample_img8], out)
def superFAN_Constrain(FAN1=None,FAN2=None,FAN3=None,SR=None):
    input_img = Input(shape=(24,24,3,),name='C1_Input')
    upsample_img2 = Input(shape=(48,48,3,),name='C1_Upsample')
    upsample_img4 = Input(shape=(96,96,3,),name='C2_Upsample')
    upsample_img8 = Input(shape=(192,192,3,),name='C3_Upsample')
    
    upsample = generator(input_img,SR)
    LR2 = Conv2D(3,3 , padding='same', kernel_initializer='he_normal',name='C1_RESIDUAL',weights=SR.get_layer('C1_RESIDUAL').get_weights())(upsample)
    LR2_ = add([LR2,upsample_img2],name='C1_RGB')
   

    for index in range(3):
        upsample = res_block_gen( upsample, 3, 64, 1,"C2_%d"%index,SR)
    LR2_FAN = create_hourglass_network(1,64,bottleneck_block,8,FAN1)(LR2)
    
    upsample = BatchNormalization(name="C2_Encode_BN",weights=SR.get_layer('C2_Encode_BN').get_weights())(upsample)
#     upsample = BatchNormalization(name="C2_Encode_BN")(upsample)
    upsample = Activation('relu',name="C2_Encode_End")(upsample)
    
    
    upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C2_Decode_End",weights=SR.get_layer("C2_Decode_End").get_weights())(upsample)
#     upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C2_Decode_End")(upsample)
    LR4 = Conv2D(3,3, padding='same', kernel_initializer='he_normal',name='C2_RESIDUAL',weights=SR.get_layer("C2_RESIDUAL").get_weights())(upsample)
    LR4_ = add([LR4,upsample_img4],name='C2_RGB')
   

    for index in range(3):
        upsample = res_block_gen( upsample, 3, 64, 1,"C3_%d"%index,SR)
    LR4_FAN = create_hourglass_network(1,64,bottleneck_block,8,FAN2)(LR4)
    
    upsample = BatchNormalization(name="C3_Encode_BN",weights=SR.get_layer('C3_Encode_BN').get_weights())(upsample)
#     upsample = BatchNormalization(name="C3_Encode_BN")(upsample)
    upsample = Activation('relu',name="C3_Encode_End")(upsample)
    
    
    
    upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C3_Decode_End",weights=SR.get_layer("C3_Decode_End").get_weights())(upsample)
#     upsample = Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "same",name="C3_Decode_End")(upsample)
    upsample = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C3_conv1",weights=SR.get_layer('C3_conv1').get_weights())(upsample)
    upsample = Activation('relu',name="C3_act1")(upsample)
    upsample = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same",name="C3_conv2",weights=SR.get_layer('C3_conv2').get_weights())(upsample)
    upsample = Activation('relu',name="C3_act2")(upsample)
    
    ##这里的conv kernel size变成了3 
    LR8 = Conv2D(3,3, padding='same', kernel_initializer='he_normal',name='C3_RESIDUAL',weights=SR.get_layer('C3_RESIDUAL').get_weights())(upsample)
    LR8_ = add([LR8,upsample_img8],name='C3_RGB')
    LR8_FAN = create_hourglass_network(1,64,bottleneck_block,8,FAN3)(LR8)
    out = []
    out.append(LR2_)
    out.append(LR4_)
    out.append(LR8_)
    out.append(LR2_FAN)
    out.append(LR4_FAN)
    out.append(LR8_FAN)
    return Model([input_img,upsample_img2,upsample_img4,upsample_img8], out)

In [7]:
model = superFAN()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C1_Input (InputLayer)           [(None, 24, 24, 3)]  0                                            
__________________________________________________________________________________________________
C1_feature_extract (Conv2D)     (None, 24, 24, 64)   1792        C1_Input[0][0]                   
__________________________________________________________________________________________________
C1_RELU (Activation)            (None, 24, 24, 64)   0           C1_feature_extract[0][0]         
__________________________________________________________________________________________________
C1_0_1conv (Conv2D)             (None, 24, 24, 64)   36928       C1_RELU[0][0]                    
____________________________________________________________________________________________

In [8]:
model = superFAN_Concat()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C1_Input (InputLayer)           [(None, 24, 24, 3)]  0                                            
__________________________________________________________________________________________________
C1_feature_extract (Conv2D)     (None, 24, 24, 64)   1792        C1_Input[0][0]                   
__________________________________________________________________________________________________
C1_RELU (Activation)            (None, 24, 24, 64)   0           C1_feature_extract[0][0]         
__________________________________________________________________________________________________
C1_0_1conv (Conv2D)             (None, 24, 24, 64)   36928       C1_RELU[0][0]                    
____________________________________________________________________________________________

In [12]:
1617033*4/(1024*1024)

6.168491363525391